# Notebook de Pedro

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Clase `myLinearRegression`

La clase implementa tres variantes del modelo de **regresión lineal**:
**OLS**, **Ridge**, y **AutoRidge**.
Todas comparten la misma estructura básica: estimar los coeficientes $\beta$ que mejor explican $y \approx X\beta + \beta_0$.

---

* **1. OLS (Ordinary Least Squares)**

**Objetivo:**
$$
\min_{\beta_0,\beta}||y - (\beta_0 + X\beta)||_2^2
$$

**Solución cerrada:**
$$
\hat{\theta} =
\begin{bmatrix}\hat{\beta}_0,\hat{\beta}\end{bmatrix}^T
(X^\top X)^{-1} X^\top y
$$

**Características algorítmicas:**

* Calcula la solución exacta mediante la ecuación normal.
* Sin ningún tipo de regularización.
* Puede ser inestable si las variables están muy correlacionadas (matriz (X^\top X) casi singular).

---

* **2. Ridge Regression (Regularización L2)**

**Objetivo:**
$$
\min_{\beta_0,\beta}; ||y - (\beta_0 + X\beta)||_2^2+\alpha ||\beta||_2^2
$$

**Solución analítica:**
$$
\hat{\theta}_{ridge} =
(X^\top X + \alpha D)^{-1} X^\top y
$$
donde $D = \mathrm{diag}(0,1,\dots,1)$ para **no penalizar el intercepto**.

**Características algorítmicas:**

* Introduce un término de penalización proporcional al cuadrado de los coeficientes.
* Atenúa la varianza y mejora la estabilidad numérica.
* El parámetro $\alpha$ controla el grado de encogimiento de los coeficientes.

---

* **3. AutoRidge (Regularización adaptativa y validación interna)**

**Objetivo:** aplicar una **penalización variable por característica** y encontrar de forma **automática** la intensidad óptima de regularización.

**Etapas algorítmicas**

1. **Penalización adaptativa por varianza**
   Cada coeficiente recibe un peso proporcional a la varianza de su variable:
   $$
   \lambda_j = \frac{\mathrm{Var}(X_j)}{\overline{\mathrm{Var}}}
   $$
   De esta forma, variables más dispersas se penalizan más.

2. **Rejilla de factores de regularización**
   Se prueban varios multiplicadores (k) alrededor de 1:
   $$
   k \in \{1-1.5\gamma, 1-\gamma, 1, 1+\gamma, 1+1.5\gamma\}
   $$
   donde $\gamma$ controla la amplitud de la búsqueda.

3. **Evaluación Leave-One-Out (LOO)**
   Para cada $k$, se calcula la matriz:
   $$
   A = X^\top X + kD
   \quad\text{y}\quad
   \hat{y} = X A^{-1} X^\top y
   $$
   Se estima el **error LOO** mediante:
   $$
   e_i^{LOO} = \frac{y_i - \hat{y}_i}{1 - h_{ii}},
   \qquad
   MSE_{LOO}(k) = \frac{1}{n}\sum_i (e_i^{LOO})^2
   $$
   El $k$ que minimiza $MSE_{LOO}$ se selecciona como óptimo.

4. **Reentrenamiento final**
   Se calcula el modelo definitivo con el mejor (k^*):
   $$
   \hat{\theta}_{final} = (X^\top X + k^* D)^{-1} X^\top y
   $$


In [ ]:
class myLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(self, method="ols", alpha=1.0, gamma=0.1):
        self.method   = method
        self.alpha    = alpha
        self.gamma    = gamma
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1, 1)

        if self.method == 'ols':
            self._fit_ols_(X, y)
        elif self.method == 'ridge':
            self._fit_ridge_(X, y)
        elif self.method == 'autoridge':
            self._fit_autoridge_(X, y)
        else:
            raise ValueError(f"Método desconocido: {self.method}")

        return self 

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return X @ self.coef_ + self.intercept_


    def _fit_ols_(self, X, y):
        X_ = np.hstack([np.ones((X.shape[0], 1)), X])
        XtX = X_.T @ X_
        Xty = X_.T @ y
        beta = np.inv(XtX) @ Xty
        self.intercept_ = float(beta[0, 0])
        self.coef_ = beta[1:, 0]

    def _fit_ridge_(self, X, y):
        X_ = np.hstack([np.ones((X.shape[0], 1)), X])
        XtX = X_.T @ X_
        Xty = X_.T @ y
        D = np.eye(X_.shape[1]); D[0, 0] = 0.0  # no penalizar intercepto
        beta = np.inv(XtX + self.alpha * D) @ Xty
        self.intercept_ = float(beta[0, 0])
        self.coef_ = beta[1:, 0]

    def _fit_autoridge_(self, X, y):
        # 1) penalización por varianza (intercepto no penalizado)
        n, p = X.shape
        X_  = np.hstack([np.ones((n, 1)), X])
        XtX = X_.T @ X_
        Xty = X_.T @ y

        var = X.var(axis=0) + 1e-12                 # (p,)
        base_lambdas = var / var.mean()             # media = 1
        D_base = np.diag(np.hstack(([0.0], base_lambdas)))  # (p+1,p+1)

        # 2) rejilla dependiendo de gamma
        k_grid = np.array([1 - 1.5*self.gamma, 1 - self.gamma,1, 1 + self.gamma, 1 + 1.5*self.gamma])

        best_k = None
        best_mse = np.inf

        for k in k_grid:
            A = XtX + k * D_base
            invA = np.linalg.inv(A)
            beta = invA @ Xty
            y_hat = X_ @ beta

            # 3) MSE LOO (PRESS): e_loo = (y - y_hat) / (1 - h_ii)
            H_diag = np.sum((X_ @ invA) * X_, axis=1)  # diag(X invA X^T)
            resid = (y - y_hat).ravel()
            e_loo = resid / (1.0 - H_diag + 1e-12)
            mse_loo = float(np.mean(e_loo**2))

            if mse_loo < best_mse:
                best_mse = mse_loo
                best_k = k
        # refit con el mejor k
        A = XtX + best_k * D_base
        beta = np.linalg.inv(A) @ Xty
        self.intercept_ = float(beta[0, 0])
        self.coef_ = beta[1:, 0]

In [41]:
datos = pd.read_csv('data/data_regresion.csv')

# Dividimos los datos en conjunto de entrenamiento y prueba
X = datos.drop('Popularity', axis=1)
y = datos['Popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


lr = myLinearRegression(method="ols")
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred_lr)
print("Mean Squared Error:", mse)

lr2 = myLinearRegression(method="ridge", alpha=0.3)
lr2.fit(X_train, y_train)
y_pred_lr2 = lr2.predict(X_test)
mse2 = mean_squared_error(y_test, y_pred_lr2)
print("Mean Squared Error (Ridge):", mse2)

l3 = myLinearRegression(method="autoridge", gamma=0.2)
l3.fit(X_train, y_train)
y_pred_lr3 = l3.predict(X_test)
mse3 = mean_squared_error(y_test, y_pred_lr3)
print("Mean Squared Error (AutoRidge):", mse3)


Mean Squared Error: 283.87722567896213
Mean Squared Error (Ridge): 283.87730541113467
Mean Squared Error (AutoRidge): 283.87757178114214
